# 0 - Are You Julish?

### Defining Values

In [1]:
# (define pie 3.14)
pie = 3.14

# (define a-radius 8.4)
a_radius = 8.4

# (define an-area (* pie (* a-radius a-radius)))
an_area = pie * a_radius * a_radius

221.5584

### Defining Functions

In [2]:
# (define area-of-circle (λ (r) (* pie (* r r))))

# Three alternatives to do that in julia...

function area_of_circle_a(r)
    pie * r * r
end

area_of_circle_b(r) = pie * r * r

area_of_circle_c = (r) -> pie * r * r

#11 (generic function with 1 method)

### Functions returning Functions

In [3]:
area_of_rectangle = (width) -> (height) -> width * height

area_of_rectangle(11)(12)

132

### Functions can take Functions

In [4]:
double_the_result_of_f = (f) -> (z) -> 2 * f(z)

add3(x) = 3 + x

dump(double_the_result_of_f(add3))
dump(double_the_result_of_f(add3)(4))

#18 (function of type var"#18#20"{typeof(add3)})
  f: add3 (function of type typeof(add3))
Int64 14


### There is no Cond

In [5]:
# We don't have cond, but if/elseif/else sequences are basically the same
if pie == 4
    28
elseif pie < 4
    33
else 
    17
end

33

In [6]:
function abs(x)
    if x < 0
        -x
    else
        x
    end
end

abs(-42) == abs(42)

true

### Julia has `let` ...but do we (always) need it?

In [7]:
function silly_abs(x)
    let x_is_negative = x < 0
        x_is_negative ? -x : x
    end
end

function silly_abs_no_let(x)
    x_is_negative = x < 0
    x_is_negative ? -x : x
end

# The compiler is too smart and realizes what we're doing
@code_llvm(silly_abs(0))
@code_llvm(silly_abs_no_let(0))

;  @ /home/martin/git-learning/the-little-learner/notebooks/chapter00.ipynb:1 within `silly_abs`
define i64 @julia_silly_abs_1201(i64 signext %0) #0 {
top:
;  @ /home/martin/git-learning/the-little-learner/notebooks/chapter00.ipynb:3 within `silly_abs`
  %1 

= call i64 @llvm.abs.i64(i64 %0, i1 false)
  ret i64 %1
}
;  @ /home/martin/git-learning/the-little-learner/notebooks/chapter00.ipynb:7 within `silly_abs_no_let`
define i64 @julia_silly_abs_no_let_1222(i64 signext %0) #0 {
top:
;  @ /home/martin/git-learning/the-little-learner/notebooks/chapter00.ipynb:9 within `silly_abs_no_let`
  %1 = call i64 @llvm.abs.i64(i64 %0, i1 false)
  ret i64 %1
}


### We still haven't seen loops

In [8]:
using BenchmarkTools

function remainder(x, y)
    if x < y
        x
    else
        remainder(x - y, y)
    end
end


# We can't rely on TCO in Julia, so it may be better to use loops

function loopy_remainder(x, y)
    while x >= y
        x -= y
    end
    x
end

display(@benchmark remainder(95124572, 723))
display(@benchmark loopy_remainder(95124572, 723))

BenchmarkTools.Trial: 5745 samples with 1 evaluation.
 Range (min … max):  653.270 μs …   2.054 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     814.945 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   864.195 μs ± 183.912 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▂ ▄▂▃▃▂▃▂ ▁                                                  
  ███████████████▇▇▇▇▅▆▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁ ▃
  653 μs           Histogram: frequency by time         1.47 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.671 ns … 69.625 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.436 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.501 ns ±  1.366 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂        ▂                  █   ▁                           
  █▁▇▁▇▂▇▅▄█▂▃█▂▆▄▂▆▂▂▄▆▃▂█▂▂▂█▂▃▂█▆▃▂▃▆▂▂▂▂▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  2.67 ns        Histogram: frequency by time        4.96 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
# Represent natural numbers as nested tuples (count the commas :)
is_zero(x) = x == ()
add1(x) = (x,)
sub1(x) =x[1]

function add(n, m)    
    result = n
    while !is_zero(m)
        m = sub1(m)
        result = add1(result)
    end
    result
end

add(((),), (((),),))

((((),),),)